In [9]:
import openai
import minsearch
import json

In [10]:
with open('documents.json','rt') as f:
    docs_raw = json.load(f)

In [11]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [12]:
index = minsearch.Index(
    text_fields=['question','text','section'],
    keyword_fields=['course']
)

In [13]:
q = 'The course already started can i still enroll'

In [14]:
index.fit(documents)

In [17]:
def search_query(query):
    boost = {'question': 3.0, 'section': 0.5}
    
    result = index.search(
    query=query,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
    )
    return result

In [21]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [22]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [23]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [17]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o",
)

In [9]:
import requests

url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json"
response = requests.get(url)

if response.status_code == 200:
    with open("documents.json", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


File downloaded successfully.
